In [33]:
import mdptoolbox
import pandas as pd
import numpy as np
import random
from math import sqrt

In [34]:
ts_df = pd.read_csv("./dataset/generated_data/ts_df.csv")
ts_df = ts_df.iloc[:,1:]

ts_df = ts_df.rename(columns={'action': 'action_0', 'action_2': 'action'})

In [35]:
def divide_CV(input, n_folds):
    data_list = list(input)
    random.shuffle(data_list) ## Randomize to avoid temporal/selection bias

    subset_size = len(data_list) // n_folds
    remainder = len(data_list) % n_folds

    subsets = []
    start_idx = 0
    ## Ensures max fold size difference ≤ 1
    for i in range(n_folds):
        end_idx = start_idx + subset_size + (1 if i < remainder else 0) ## Folds get 1 extra patient from remainder
        subsets.append(data_list[start_idx:end_idx])
        start_idx = end_idx
    return subsets

#define the reward function
def Reward(a, b, mode):
    R = np.zeros(12).reshape(4,3)
    if mode == "reward":
        R[0,:] += 1
    else:
        R[3,:] -= 1
    R[:,1] -= a
    R[:,2] -= b

    return R

"""
Reward Mode

actions = [maintain, subtle increase, increase]
R = [
    [ 1,   1-a,   1-b ],  # Cluster 0 (Fair State)
    [ 0,   -a,    -b ],    # Cluster 1 (Worsening State)
    [ 0,   -a,    -b ],    # Cluster 2 (Motor-focused Worsening State)
    [ 0,   -a,    -b ]   # Cluster 3 (Deteriorating State)
]

Penalty Mode
R = [
    [ 0,   -a,    -b ],  # Cluster 0 (Fair State)
    [ 0,   -a,    -b ],    # Cluster 1 (Worsening State)
    [ 0,   -a,    -b ],    # Cluster 2 (Motor-focused Worsening State)
    [ -1,  -1-a,  -1-b ]   # Cluster 3 (Deteriorating State)
]

"""

'\nReward Mode\n\nactions = [maintain, subtle increase, increase]\nR = [\n    [ 1,   1-a,   1-b ],  # Cluster 0 (Fair State)\n    [ 0,   -a,    -b ],    # Cluster 1 (Worsening State)\n    [ 0,   -a,    -b ],    # Cluster 2 (Motor-focused Worsening State)\n    [ 0,   -a,    -b ]   # Cluster 3 (Deteriorating State)\n]\n\nPenalty Mode\nR = [\n    [ 0,   -a,    -b ],  # Cluster 0 (Fair State)\n    [ 0,   -a,    -b ],    # Cluster 1 (Worsening State)\n    [ 0,   -a,    -b ],    # Cluster 2 (Motor-focused Worsening State)\n    [ -1,  -1-a,  -1-b ]   # Cluster 3 (Deteriorating State)\n]\n\n'

In [36]:
def calculate_transition_probabilities(data, num_actions=3, num_states=4):
    """Calculate transition probabilities from data"""
    P = np.zeros((num_actions, num_states, num_states))
    
    for a in range(num_actions):
        for s in range(num_states):
            for s_ in range(num_states - 1):  # Last state is absorbing
                criteria = ((data['cluster'] == s) & (data['action'] == a))
                criteria_next = criteria & (data['cluster_n'] == s_)
                P[a, s, s_] = criteria_next.sum() / criteria.sum() if criteria.sum() > 0 else 0
            
            # Probability of staying in absorbing state
            P[a, s, num_states-1] = 1 - sum(P[a, s, :num_states-1])
    
    return P

In [37]:
def run_mdp_models(P, reward_params):
    """Run different MDP models with given transition matrix and reward parameters"""
    a, b = reward_params
    R_r = Reward(a, b, "reward")
    R_p = Reward(a, b, "penalty")
    
    # Policy iteration models
    mdp_model_r = mdptoolbox.mdp.PolicyIteration(P, R_r, 0.99)
    mdp_model_p = mdptoolbox.mdp.PolicyIteration(P, R_p, 0.99)
    mdp_model_r_w = mdptoolbox.mdp.PolicyIteration(P, -R_r, 0.99)
    mdp_model_p_w = mdptoolbox.mdp.PolicyIteration(P, -R_p, 0.99)
    
    # Value iteration models
    vi_model_r = mdptoolbox.mdp.ValueIteration(P, R_r, 0.99)
    vi_model_p = mdptoolbox.mdp.ValueIteration(P, R_p, 0.99)

    vi_model_r_w = mdptoolbox.mdp.ValueIteration(P, -R_r, 0.99)
    vi_model_r_w.run()

    vi_model_p_w = mdptoolbox.mdp.ValueIteration(P, -R_p, 0.99)
    vi_model_p_w.run()
    
    # Run all models
    for model in [mdp_model_r, mdp_model_p, mdp_model_r_w, mdp_model_p_w, vi_model_r, vi_model_p]:
        model.run()
    # for model in [mdp_model_r, mdp_model_p, mdp_model_r_w, mdp_model_p_w]:
    #     model.run()
    
    # return {
    #     "r":mdp_model_r.policy,
    #     "p":mdp_model_p.policy,
    #     "r_w" : mdp_model_r_w.policy,
    #     "p_w" : mdp_model_r_w.policy,
        
    #     }
    return {
            "r":mdp_model_r.policy,
            "p":mdp_model_p.policy,
            "r_w" : mdp_model_r_w.policy,
            "p_w" : mdp_model_r_w.policy,
            "vi_r": vi_model_r.policy,
            "vi_p": vi_model_p.policy,
            "vi_r_w": vi_model_r_w.policy,
            "vi_p_w": vi_model_p_w.policy
            }


In [38]:
def simulate_patient_traj(pat_data, policies, P_v, reward_params, tests, actions_chosen):
    
    init_LEDD = list(pat_data['LEDD'])[0]
    
    # LEDD = {"optimal":[init_LEDD,init_LEDD],
    #         "random":[init_LEDD,init_LEDD],
    #         "const":[init_LEDD,init_LEDD],
    #         "worst":[init_LEDD,init_LEDD],
    #         "real":[0,0],
    #         "vi": [init_LEDD, init_LEDD],
    #         "vi_worst": [init_LEDD, init_LEDD]} #sim_r, sim_p, worst_r, worst_p, constant, random
    
    # LEDD_traj = {"optimal":[init_LEDD,init_LEDD],
    #         "random":[init_LEDD,init_LEDD],
    #         "const":[init_LEDD,init_LEDD],
    #         "worst":[init_LEDD,init_LEDD],
    #         "real":[0,0],
    #         "vi":[init_LEDD, init_LEDD],
    #         "vi_worst":[init_LEDD, init_LEDD]}

    # state = pat_data['cluster']
    # state = [list(state)[0],list(state)[0]]

    rewards = {
        "optimal": [0,0],
        "worst": [0,0],
        "const": [0,0],
        "random": [0,0],
        "real": [0,0],
        "vi": [0,0],
        "vi_worst": [0,0]
    }

    state = [pat_data['cluster'].iloc[0], pat_data['cluster'].iloc[0]]

    # LEDD_total["real"].append(pat_data["LEDD"].mean())

    pat_data_len = len(pat_data)

    state = [pat_data['cluster'].iloc[0], pat_data['cluster'].iloc[0]]
    
    # LEDD_total["real"].append(pat_data["LEDD"].mean())
    
    pat_data_len = len(pat_data)

    #simul
    for t in tests:
        if state[0] == 0:
            rewards[t][0] += 1
        elif state[1] == 3:
            rewards[t][1] -= 1
        for i in range(pat_data_len - 1):
            # Determine actions for each test type
            actions = {
                "optimal": [policies['r'][state[0]], policies['p'][state[1]]],
                "worst": [policies['r_w'][state[1]], policies['p_w'][state[1]]],
                "vi": [policies['vi_r'][state[0]], policies['vi_p'][state[1]]],
                "vi_worst": [policies['vi_r_w'][state[0]], policies['vi_p_w'][state[1]]],
                "const": [0, 0],            
                "real": [pat_data['action'].iloc[i], pat_data['action'].iloc[i]],
                "random": [random.choice(range(3)), random.choice(range(3))]
            }
           
            for i in range(2):
               actions_chosen[t][i].append(actions[t][i])

            for k in range(2):
                if actions[t][k] == 1:
                    rewards[t][k] -= 0.01
                elif actions[t][k] == 2:
                    rewards[t][k] -= 0.025

            if state[0] == 0:
                rewards[t][0] += 1
            elif state[1] == 3:
                rewards[t][1] -= 1

            state = [
                random.choices(range(4), weights=P_v[int(actions[t][0]), int(state[0]), :])[0],
                random.choices(range(4), weights=P_v[int(actions[t][1]), int(state[1]), :])[0]
            ]
        
        # LEDD_traj[t][0] /= pat_data_len
        # LEDD_traj[t][1] /= pat_data_len

        # LEDD_total[t][0].append(LEDD_traj[t][0])
        # LEDD_total[t][1].append(LEDD_traj[t][1])

    return rewards


    # sim_reward = [s/tot_data_len for s in sim_reward]
    # sim_reward_w = [s/tot_data_len for s in sim_reward_w]
    # const_reward = [s/tot_data_len for s in const_reward]
    # random_reward = [s/tot_data_len for s in random_reward]
    # real_reward = [s/tot_data_len for s in real_reward]
    # vi_reward = [s/tot_data_len for s in vi_reward]
    # vi_reward_w = [s/tot_data_len for s in vi_reward_w]

In [ ]:

tot_sim, tot_const, tot_random, tot_sim_w, tot_real, tot_vi, tot_vi_w = [[],[]], [[],[]], [[],[]], [[],[]], [[],[]], [[],[]], [[],[]]
LEDD_total = [0 for _ in range(8)] #sim_r, sim_w, worst_p, worst_p, constant, random, actual 
LEDD_total = {"optimal":[[],[]],
                "random":[[],[]],
                "const":[[],[]],
                "worst":[[],[]],
                "real":[[],[]],
                "vi": [[],[]],
                "vi_worst": [[],[]]}
tests = ["optimal", "random", "const", "worst", "real"]
# fix the below error


def validate_model(ts_df, subsets, n_folds, reward_params=(0.01, 0.025), tests=None):
    tests = ["optimal", "random", "const", "worst", "real", "vi", "vi_worst"]
    # tests = ["optimal", "random", "const", "worst", "real"]
    actions_chosen = {
    t: [[],[]] for t in tests
    }
    folds_rewards = {
        "optimal": [[], []],
        "random": [[], []],
        "const": [[], []],
        "worst": [[], []],
        "real": [[], []],
        "vi": [[], []],
        "vi_worst": [[], []]
    }


    for i in range(n_folds + 1):
        if i < n_folds:
            tot_rewards = {
                "optimal": [0, 0],
                "worst": [0, 0],
                "vi": [0, 0],
                "vi_worst": [0, 0],
                "const": [0, 0],
                "random": [0, 0],
                "real": [0, 0]
            }

            valid_idx = subsets[i]
            train_idx = []
            for j in range(n_folds):
                if i != j:
                    train_idx += subsets[j]

            valid = ts_df[ts_df['PATNO'].isin(valid_idx)]
            train = ts_df[ts_df['PATNO'].isin(train_idx)]
        else:
            valid = ts_df


        P_t = calculate_transition_probabilities(train)
        P_v = calculate_transition_probabilities(ts_df)

        # Run MDP models
        policies = run_mdp_models(P_t, reward_params)
        
        tot_data_len = 0
        for pat in valid_idx:
            pat_data = valid[valid['PATNO'] == pat]

            if pat_data.empty:
                continue

            tot_data_len += len(pat_data)

            patient_rewards = simulate_patient_traj(pat_data, policies, P_v, reward_params, tests, actions_chosen)

            for t in tests:
                for k in range(2):
                    tot_rewards[t][k] += patient_rewards[t][k]

        
        if tot_data_len > 0:
            for t in tests:
                for k in range(2):
                    tot_rewards[t][k] /= tot_data_len

        for t in tests:
            for k in range(2):
                folds_rewards[t][k].append(tot_rewards[t][k])
    
    num_actions = {
        "optimal": [],
        "random": [],
        "const": [],
        "worst": [],
        "real": [],
        "vi": [],
        "vi_worst": []
    }
    for t in tests:
        num_actions[t].append(actions_chosen[t][0].count(0))
        num_actions[t].append(actions_chosen[t][0].count(1))
        num_actions[t].append(actions_chosen[t][0].count(2))


    # modes = ["reward", "penalty"]
    # for i in range(2):
    #     print(modes[i].upper())
    #     print("opti",  "\t", np.round(np.mean(folds_rewards["optimal"][i]),4), "\t", np.round(np.std(folds_rewards["optimal"][i])/sqrt(n_folds),4))
    #     print("worst", "\t", np.round(np.mean(folds_rewards["worst"][i]),4), "\t", np.round(np.std(folds_rewards["worst"][i])/sqrt(n_folds),4))
    #     print("const",  "\t", np.round(np.mean(folds_rewards["const"][i]),4),  "\t", np.round(np.std(folds_rewards["const"][i])/sqrt(n_folds),4))
    #     print("random",  "\t", np.round(np.mean(folds_rewards["random"][i]),4), "\t", np.round(np.std(folds_rewards["random"][i])/sqrt(n_folds),4))
    #     print("VI",  "\t", np.round(np.mean(folds_rewards["vi"][i]),4), "\t", np.round(np.std(folds_rewards["vi"][i])/sqrt(n_folds),4))
    #     print("VI w",  "\t", np.round(np.mean(folds_rewards["vi_worst"][i]),4), "\t", np.round(np.std(folds_rewards["vi_worst"][i])/sqrt(n_folds),4))
    #     print("real \t", np.round(np.mean(folds_rewards["real"][i]),4))
    #     print()
    
    return [folds_rewards, num_actions]


In [ ]:
n_folds = 10

patients = ts_df.iloc[:,0]
patients = set(patients)

test_results = {
        "optimal": [[], []],
        "random": [[], []],
        "const": [[], []],
        "worst": [[], []],
        "real": [[], []],
        "vi": [[], []],
        "vi_worst": [[], []]
    }

for i in range(10):
    subsets = divide_CV(patients, n_folds)
    results, num_actions = validate_model(ts_df, subsets, n_folds)

    all_num_actions = {
        "optimal": [[], [], []],
        "random": [[], [], []],
        "const": [[], [], []],
        "worst": [[], [], []],
        "real": [[], [], []],
        "vi": [[], [], []],
        "vi_worst": [[], [], []]
    }

    for t in all_num_actions.keys():
        for i in range(3):
            all_num_actions[t][i].append(num_actions[t][i])


#     for j in range(2):
#         test_results['optimal'][j].append(np.round(np.mean(results["optimal"][j]),4))
#         test_results['worst'][j].append(np.round(np.mean(results["worst"][j]),4))
#         test_results['const'][j].append(np.round(np.mean(results["const"][j]),4))
#         test_results['random'][j].append(np.round(np.mean(results["random"][j]),4))
#         test_results['vi'][j].append(np.round(np.mean(results["vi"][j]),4))
#         test_results['vi_worst'][j].append(np.round(np.mean(results["vi_worst"][j]),4))
#         test_results['real'][j].append(np.round(np.mean(results["real"][j]),4))

print("ACTIONS CHOSEN ACROSS ALL 10 RUNS:")
for t in all_num_actions.keys():
    print(f"`Actions chosen for {t} : 0s={np.mean(all_num_actions[t][0])}, 1s={np.mean(all_num_actions[t][1])}, 2s={np.mean(all_num_actions[t][2])}`")
print()

# modes = ["reward", "penalty"]
# for i in range(2):
#     print(modes[i].upper())
#     print("opti",  "\t", np.round(np.mean(test_results["optimal"][i]),4), "\t", np.round(np.std(test_results["optimal"][i])/sqrt(n_folds),4))
#     print("worst", "\t", np.round(np.mean(test_results["worst"][i]),4), "\t", np.round(np.std(test_results["worst"][i])/sqrt(n_folds),4))
#     print("const",  "\t", np.round(np.mean(test_results["const"][i]),4),  "\t", np.round(np.std(test_results["const"][i])/sqrt(n_folds),4))
#     print("random",  "\t", np.round(np.mean(test_results["random"][i]),4), "\t", np.round(np.std(test_results["random"][i])/sqrt(n_folds),4))
#     print("VI",  "\t", np.round(np.mean(test_results["vi"][i]),4), "\t", np.round(np.std(test_results["vi"][i])/sqrt(n_folds),4))
#     print("VI w",  "\t", np.round(np.mean(test_results["vi_worst"][i]),4), "\t", np.round(np.std(test_results["vi_worst"][i])/sqrt(n_folds),4))
#     print("real \t", np.round(np.mean(test_results["real"][i]),4))
#     print()

REWARD
opti 	 0.3451 	 0.0033
worst 	 0.2961 	 0.003
const 	 0.3194 	 0.0017
random 	 0.3082 	 0.0021
VI 	 0.3261 	 0.0024
VI w 	 0.2846 	 0.002
real 	 0.3064

PENALTY
opti 	 -0.1262 	 0.001
worst 	 -0.1372 	 0.0018
const 	 -0.126 	 0.0014
random 	 -0.135 	 0.0013
VI 	 -0.1242 	 0.0017
VI w 	 -0.1583 	 0.0025
real 	 -0.133

